<a href="https://colab.research.google.com/github/goya5858/commonlitreadabilityprize/blob/main/working/RoBERTa_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install libs

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
compe_name = 'commonlitreadabilityprize'
%cd /content/drive/MyDrive/kaggle/works/$compe_name/working/

!pip install -q -q -q -U albumentations
!pip install -q -q -q -U torch
!pip install -q -q -q timm
!pip install -q -q -q pytorch_lightning
!pip install -q -q -q -U transformers
!pip install -q -q -q -U sentencepiece

import os
import re
import gc
import sys
import time
import copy
import pickle
import random
import warnings
from tqdm import tqdm_notebook as tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import cv2
import PIL.Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations
from albumentations.pytorch.transforms import ToTensorV2

import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import CosineAnnealingLR

import timm

import nltk
from wordcloud import WordCloud
nltk.download('stopwords')
from nltk.corpus import stopwords
stop=stopwords.words('english')
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob,Word
from collections import Counter
import string
from torch.nn.utils.rnn import pad_sequence

from torch.nn import MSELoss

from torch.cuda import amp

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import model_selection

import transformers
from transformers import get_linear_schedule_with_warmup, AdamW

from transformers import *

def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    #tf.random.set_seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

/content/drive/MyDrive/kaggle/works/commonlitreadabilityprize/working
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Dataset

In [3]:
DEVICE = 'cuda:0'

get_tokenizer = RobertaTokenizer
#get_model     = BartForSequenceClassification
#get_tokenizer = AutoTokenizer #RobertaTokenizer より汎用性ある気がする　使い勝手的な面で
#get_model     = AutoModel
#get_model     = RobertaForSequenceClassification
get_model     = RobertaModel

VOCAB_PATH = 'roberta-base'
MODEL_PATH = 'roberta-base'
#VOCAB_PATH = 'bert-base-uncased'
#MODEL_PATH = 'bert-base-uncased'

In [4]:
ROOT = f"../input/{compe_name}/"
MODEL_ROOT = "../input/models/RoBERTa_small"

df = pd.read_csv(ROOT+'train.csv')
df.head()

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845


In [5]:
test_df = pd.read_csv(ROOT+'test.csv')
test_df.head()

,id,url_legal,license,excerpt
0,c0f722661,NaN,NaN,My hope lay in Jack's promise that he would ke...
1,f0953f0a5,NaN,NaN,Dotty continued to go to Mrs. Gray's every nig...
2,0df072751,NaN,NaN,It was a bright and cheerful scene that greete...
3,04caf4e0c,https://en.wikipedia.org/wiki/Cell_division,CC BY-SA 3.0,Cell division is the process by which a parent...
4,0e63f8bea,https://en.wikipedia.org/wiki/Debugging,CC BY-SA 3.0,Debugging is the process of finding and resolv...


In [6]:
def prep_text(text_df):
    text_df = text_df.str.replace("\n","",regex=False)
    return text_df.str.replace("\'s",r"s",regex=True).values

df['excerpt']      = prep_text(df['excerpt'])
test_df['excerpt'] = prep_text(test_df['excerpt'])

MAX_SEQUENCE_LENGTH = df['excerpt'].apply(lambda x: len(x.split())).max()

tokenizer = get_tokenizer.from_pretrained(VOCAB_PATH,
                                          model_max_length=MAX_SEQUENCE_LENGTH
                                          )
df['token']          = df['excerpt'].apply(tokenizer)
test_df['token']   = test_df['excerpt'].apply(tokenizer)

Token indices sequence length is longer than the specified maximum sequence length for this model (222 > 205). Running this sequence through the model will result in indexing errors


In [7]:
RoBERTa_pred = pd.read_csv(ROOT+'RoBERTa_oofs.csv')
RoBERTa_pred

,seed_0,seed_7,seed_42,seed_88,seed_100,target
0,-0.259451,-0.062605,0.079190,-0.320383,-0.210586,-0.340259
1,-0.434118,-0.200020,-0.262286,-0.051768,-0.131072,-0.315372
2,-0.364411,-0.639652,-0.425685,-0.624188,-0.569017,-0.580118
3,-1.390543,-1.851582,-1.599446,-1.322094,-1.352633,-1.054013
4,0.206669,0.280685,0.082368,0.317499,0.263067,0.247197
...,...,...,...,...,...,...
2829,0.573117,0.539531,0.275756,0.464814,0.496079,1.711390
2830,-0.201056,0.321773,0.230156,-0.000241,0.137118,0.189476
2831,-0.321227,-0.451728,-0.603753,-0.653499,-0.490357,0.255209
2832,-1.136917,-1.241406,-1.173482,-1.164049,-0.858960,-0.215279


In [8]:
class CLPDataset(Dataset):
    def __init__(self, df):
        super().__init__()
        self.token  = df.token
        self.labels = df.target

    def __len__(self):
        return self.labels.shape[0]
  
    def __getitem__(self, idx):
        if len(self.token.iloc[0]) == 2:
            return (
                    torch.tensor(self.token.iloc[idx].input_ids), \
                    #torch.tensor(self.token.iloc[idx].token_type_ids), \
                    torch.tensor(self.token.iloc[idx].attention_mask)
                  ), \
                  torch.tensor(self.labels.iloc[idx])
        if len(self.token.iloc[idx]) == 3:
            return (
                    torch.tensor(self.token.iloc[idx].input_ids), \
                    torch.tensor(self.token.iloc[idx].token_type_ids), \
                    torch.tensor(self.token.iloc[idx].attention_mask)
                    ), \
                  torch.tensor(self.labels.iloc[idx])

In [9]:
def collate_fn(batch):
    inputs, labels = zip(*batch)
    try:
        ids, types, masks = zip(*inputs)
        ids   = pad_sequence(ids, batch_first=True).to(DEVICE)
        types = pad_sequence(types, batch_first=True).to(DEVICE)
        masks = pad_sequence(masks, batch_first=True).to(DEVICE)
        labels= torch.tensor(labels, dtype=torch.float).to(DEVICE)
        return {
                    "input_ids"      : ids, \
                    "token_type_ids" : types, \
                    "attention_mask" : masks
                  }, \
                  labels
    except ValueError:
        ids, masks = zip(*inputs)
        ids   = pad_sequence(ids, batch_first=True).to(DEVICE)
        #types = pad_sequence(types, batch_first=True).to(DEVICE)
        masks = pad_sequence(masks, batch_first=True).to(DEVICE)
        labels= torch.tensor(labels, dtype=torch.float).to(DEVICE)
        return {
                    "input_ids"      : ids, \
                    #"token_type_ids" : types, \
                    "attention_mask" : masks
                  }, \
                  labels

In [10]:
# trainデータを、targetの値をビニングした値を元に層化fold
def create_folds(data, num_splits):
    # we create a new column called kfold and fill it with -1
    folds = pd.DataFrame( np.ones( (data.shape[0],1) )*-1, columns=['kfold'] )
    num_bins = int(np.floor(1 + np.log2(len(data))))

    bins = pd.cut(
          data["target"], bins=num_bins, labels=False
          )

    kf = model_selection.StratifiedKFold(n_splits=num_splits, shuffle=True)
    for f, (t_, v_) in enumerate(kf.split(X=data, y=bins)):
        folds.iloc[v_] = int(f)

    return folds

In [11]:
def get_dataloaders(df, folds, n_fold, BATCH_SIZE):
    #folds = create_folds(df, num_splits=NUM_FOLDS)
    train_df = df[(folds['kfold']!=n_fold)]
    valid_df = df[(folds['kfold']==n_fold)]

    train_dataset = CLPDataset(df=train_df)
    valid_dataset = CLPDataset(df=valid_df)

    train_loader = DataLoader(
        dataset = train_dataset,
        batch_size = BATCH_SIZE,
        shuffle = True,
        collate_fn=collate_fn,
        #num_workers = -1
        )
    valid_loader = DataLoader(
        dataset = valid_dataset,
        batch_size = BATCH_SIZE,
        shuffle = False,
        collate_fn=collate_fn,
        #num_workers = -1
        )
    return train_loader, valid_loader

# SVM

In [18]:
#import lightgbm as lgb
RoBERTa_pred = pd.read_csv('/content/drive/MyDrive/kaggle/works/commonlitreadabilityprize/input/commonlitreadabilityprize/RoBERTa_oofs.csv')
from sklearn.svm import SVR
data_cols = ['seed_0', 'seed_7', 'seed_42', 'seed_88', 'seed_100']
target_col = 'target'

def get_dataset(df, folds, n_fold):
    trn_df = df[(folds['kfold']!=n_fold)]
    val_df = df[(folds['kfold']==n_fold)]
    return trn_df[data_cols], trn_df[['target']], val_df[data_cols], val_df[['target']]

In [23]:
def loss_fn(pred, labels):
  return MSELoss()(pred.view(-1), labels.view(-1))

def train_fold(folds, n_fold, seed):
    trn_X, trn_y, val_X, val_y = get_dataset(RoBERTa_pred, folds, n_fold)
    SVM_model = SVR(kernel='rbf', degree=3, gamma='scale', C=1)
    SVM_model.fit(trn_X, trn_y)
    pred_fold = SVM_model.predict(val_X)
    model_path = f"../input/models/RoBERTa_model/seed-{seed}-fold-{n_fold}-svmmodel.sav"
    pickle.dump(SVM_model, open(model_path, 'wb'))
    del SVM_model
    gc.collect()
    return pred_fold

In [17]:
def train_seed(seed):
    seed_everything(seed)
    folds = create_folds(df, num_splits=NUM_FOLDS)
    oof   = np.zeros(df['target'].shape)

    for n_fold in range(NUM_FOLDS):
        best_pred = train_fold(folds, n_fold, seed)
        oof[(folds['kfold']==n_fold)] = best_pred
        print('-='*20)

    print('%'*50)
    oof_score = loss_fn(pred   = torch.tensor(oof).to(DEVICE),
                                  labels = torch.tensor(df['target'].values).to(DEVICE) )
    print(f'oof_score_{seed} :', np.sqrt( oof_score.cpu().detach().numpy()))
    return np.sqrt( oof_score.cpu().detach().numpy() )

In [24]:
SEEDs = [0, 7, 42, 88, 100]
NUM_FOLDS = 5

MAX_WORDS = df["excerpt"].apply(lambda x: len(x.split())).max()

oof_scores  = []
for seed in SEEDs:
  print(f'--------------- SEED {seed} is set ---------------')
  oof_score_for_seed = train_seed(seed)
  oof_scores.append(oof_score_for_seed)

print('&%&%'*30)
print("all_oof_score_avg : ", np.mean(oof_scores) )
print("all_oof_scores : ", oof_scores)

--------------- SEED 0 is set ---------------


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


...

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
oof_score_100 : 0.4877629076949275
&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%&%
all_oof_score_avg :  0.48799640685927426
all_oof_scores :  [0.4872857107368228, 0.48826032996551727, 0.4890508148188624, 0.487622271080241, 0.4877629076949275]
